# Metclasses & properties

In [ ]:
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'  # always print last expr.
%config InlineBackend.figure_format = 'svg'
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

np.set_printoptions(precision=4, floatmode="fixed", suppress=True)
rng = np.random.default_rng()

In [ ]:
from typing import Any

In [ ]:
from django.utils.functional import classproperty

In [ ]:
class ClassProperty(property):
    def __get__(self, cls, owner):
        return self.fget.__get__(None, owner)()

In [ ]:
from abc import ABC, ABCMeta, abstractmethod


class MyMetaClass(ABCMeta):
    @classmethod
    @property
    def expensive_metaclass_property(cls):
        """This may take a while to compute!"""
        print("computing metaclass property")
        return "Phew, that was a lot of work!"


class MyBaseClass(ABC, metaclass=MyMetaClass):
    @classmethod
    @property
    def expensive_class_property(cls):
        """This may take a while to compute!"""
        print("computing class property")
        return "Phew, that was a lot of work!"

    @property
    def expensive_instance_property(self):
        """This may take a while to compute!"""
        print("computing instance property")
        return "Phew, that was a lot of work!"


class MyClass(MyBaseClass):
    """Some subclass of MyBaseClass"""

In [ ]:
help(MyBaseClass())

In [ ]:
class class_reify(object):
    def __init__(self, wrapped):
        self.wrapped = wrapped
        try:
            self.__doc__ = wrapped.__doc__
        except:  # pragma: no cover
            pass

    # original sets the attributes on the instance
    # def __get__(self, inst, objtype=None):
    #    if inst is None:
    #        return self
    #    val = self.wrapped(inst)
    #    setattr(inst, self.wrapped.__name__, val)
    #    return val

    # ignore the instance, and just set them on the class
    # if called on a class, inst is None and objtype is the class
    # if called on an instance, inst is the instance, and objtype
    # the class
    def __get__(self, inst, objtype=None):
        # ask the value from the wrapped object, giving it
        # our class
        val = self.wrapped(objtype)

        # and set the attribute directly to the class, thereby
        # avoiding the descriptor to be called multiple times
        setattr(objtype, self.wrapped.__name__, val)

        # and return the calculated value
        return val


class Test(object):
    @class_reify
    def foo(cls):
        print("foo called for class", cls)
        return 42


print(Test.foo)
print(Test.foo)

In [ ]:
help(Test)

# Pure python classmethod and property

In [ ]:
class Property:
    "Emulate PyProperty_Type() in Objects/descrobject.c"

    def __init__(self, fget=None, fset=None, fdel=None, doc=None, **kwargs):
        self.fget = fget
        self.fset = fset
        self.fdel = fdel
        if doc is None and fget is not None:
            doc = fget.__doc__
        self.__doc__ = doc

    def __get__(self, obj, objtype=None):
        if obj is None:
            return self
        if self.fget is None:
            raise AttributeError("unreadable attribute")
        return self.fget(obj)

    def __set__(self, obj, value):
        if self.fset is None:
            raise AttributeError("can't set attribute")
        self.fset(obj, value)

    def __delete__(self, obj):
        if self.fdel is None:
            raise AttributeError("can't delete attribute")
        self.fdel(obj)

    def getter(self, fget):
        return type(self)(fget, self.fset, self.fdel, self.__doc__)

    def setter(self, fset):
        return type(self)(self.fget, fset, self.fdel, self.__doc__)

    def deleter(self, fdel):
        return type(self)(self.fget, self.fset, fdel, self.__doc__)

In [ ]:
class ClassMethod(object):
    "Emulate PyClassMethod_Type() in Objects/funcobject.c"

    def __init__(self, f):
        self.f = f

    def __get__(self, obj, klass=None):
        if klass is None:
            klass = type(obj)

        def newfunc(*args):
            return self.f(klass, *args)

        return newfunc

In [ ]:
class Testit:
    @classmethod
    def clsmethod(cls):
        """Real Classmethod"""
        return f"My name is {cls.__name__}"

    @ClassMethod
    def myclsmethod(cls):
        """My Classmethod"""
        return f"My name is {cls.__name__}"

    @Property
    def myprop(self):
        """My property"""
        return 42

    @property
    def prop(self):
        """Real property"""
        return 42


print(Testit().prop)
help(Testit)

## Better ClassMethod with functool.wraps

In [ ]:
from functools import wraps


class ClassMethod(object):
    "Emulate PyClassMethod_Type() in Objects/funcobject.c"

    def __init__(self, func, *args, **kwargs):
        print(
            func,
            args,
        )
        self.func = func

    def __get__(self, obj, klass=None):
        if klass is None:
            klass = type(obj)

        @wraps(self.func)
        def wrapper(*args, **kwargs):
            return self.func(klass, *args, **kwargs)

        return wrapper

In [ ]:
class Testit:
    @classmethod
    def clsmethod(cls):
        """Real Classmethod"""
        return f"My name is {cls.__name__}"

    @ClassMethod
    def myclsmethod(cls):
        """My Classmethod"""
        return f"My name is {cls.__name__}"

    @Property
    def myprop(self):
        """My property"""
        return 42

    @property
    def prop(self):
        """Real property"""
        return 42


print(Testit().prop)
help(Testit)

## New-ish classmethod descriptor

In [ ]:
from types import MethodType


class ClassMethod:
    "Emulate PyClassMethod_Type() in Objects/funcobject.c"

    def __init__(self, func):
        self.func = func

    def __get__(self, obj, cls=None):
        if cls is None:
            cls = type(obj)
        if hasattr(type(self.func), "__get__"):
            return self.func.__get__(cls)
        return MethodType(self.f, cls)

In [ ]:
class Testit:
    @classmethod
    def clsmethod(cls):
        """Real Classmethod"""
        return f"My name is {cls.__name__}"

    @ClassMethod
    @property
    def myclsmethod(cls):
        """My Classmethod"""
        return f"My name is {cls.__name__}"

    @Property
    def myprop(self):
        """My property"""
        return 42

    @property
    def prop(self):
        """Real property"""
        return 42

    def mymethod(self):
        """Instance Method"""
        return 32


print(Testit().prop)
help(Testit)

## Custom Class property

In [ ]:
from types import MethodType


class ClassProperty:
    """Lazyly evaluated attribute"""

    def __init__(self, fget=None, fset=None, fdel=None, doc=None, **kwargs):
        self.fget = fget
        self.fset = fset
        self.fdel = fdel
        if doc is None and fget is not None:
            doc = fget.__doc__
        self.__doc__ = doc

    def __get__(self, obj, objtype=None, klass=None):
        print(self.fget, self.fget(type(obj)), hasattr(type(self.fget), "__get__"))

        # if klass is None:
        #     klass = type(obj)
        # if hasattr(type(self.fget), '__get__'):
        #     return self.fget.__get__(klass).__call__()
        # return MethodType(self.fget, klass)
        print(f"{obj=}")
        if obj is None:
            if hasattr(type(self.fget), "__get__"):
                return self.fget.__get__()
        if self.fget is None:
            raise AttributeError("unreadable attribute")
        if klass is None:
            klass = type(obj)
        if hasattr(type(self.fget), "__get__"):
            print("HERE")
            return self.fget.__get__(klass).__call__()
        print("HERE")

        return self.fget(klass)

    def __set__(self, obj, value):
        if self.fset is None:
            raise AttributeError("can't set attribute")
        self.fset(obj, value)

    def __delete__(self, obj):
        if self.fdel is None:
            raise AttributeError("can't delete attribute")
        self.fdel(obj)

    def getter(self, fget):
        return type(self)(fget, self.fset, self.fdel, self.__doc__)

    def setter(self, fset):
        return type(self)(self.fget, fset, self.fdel, self.__doc__)

    def deleter(self, fdel):
        return type(self)(self.fget, self.fset, fdel, self.__doc__)

In [ ]:
class Testit:
    @classmethod
    @property
    def clsprop(cls):
        """Real Class-Property"""
        return f"My name is {cls.__name__}"

    @ClassProperty
    def myclsprop(cls):
        """My Class-Property"""
        return f"My name is {cls.__name__}"

    @Property
    def myprop(self):
        """My property"""
        return self.__class__.__name__

    @property
    def prop(self):
        """Real property"""
        return self.__class__.__name__


# print(Testit.clsprop, Testit.myclsprop)
# help(Testit)

In [ ]:
dir(Testit.__dict__["myclsprop"])

## Better class-property with inheritance

In [ ]:
class ClassProperty(property):
    """Lazyly evaluated attribute"""

    def __get__(self, obj, objtype=None, klass=None):
        if obj is None:
            return self
        if self.fget is None:
            raise AttributeError("unreadable attribute")
        if klass is None:
            klass = type(obj)
        print(klass)
        return self.fget(klass)

In [ ]:
class Testit:
    @classmethod
    @property
    def clsprop(cls):
        """Real Class-Property"""
        return f"My name is {cls.__name__}"

    @ClassProperty
    def myclsprop(cls):
        """My Class-Property"""
        return f"My name is {cls.__name__}"

    @Property
    def myprop(self):
        """My property"""
        return 42

    @property
    def prop(self):
        """Real property"""
        return 42


print(Testit.myclsprop)
help(Testit)

In [ ]:
class Property:
    "Emulate PyProperty_Type() in Objects/descrobject.c"

    def __init__(self, fget=None, fset=None, fdel=None, doc=None, **kwargs):
        self.fget = fget
        self.fset = fset
        self.fdel = fdel
        if doc is None and fget is not None:
            doc = fget.__doc__
        self.__doc__ = doc

    def __get__(self, obj, objtype=None):
        if obj is None:
            return self
        if self.fget is None:
            raise AttributeError("unreadable attribute")
        return self.fget(obj)

    def __set__(self, obj, value):
        if self.fset is None:
            raise AttributeError("can't set attribute")
        self.fset(obj, value)

    def __delete__(self, obj):
        if self.fdel is None:
            raise AttributeError("can't delete attribute")
        self.fdel(obj)

    def getter(self, fget):
        return type(self)(fget, self.fset, self.fdel, self.__doc__)

    def setter(self, fset):
        return type(self)(self.fget, fset, self.fdel, self.__doc__)

    def deleter(self, fdel):
        return type(self)(self.fget, self.fset, fdel, self.__doc__)

In [ ]:
class ClassMethod(object):
    "Emulate PyClassMethod_Type() in Objects/funcobject.c"

    def __init__(self, f):
        self.f = f

    def __get__(self, obj, klass=None):
        if klass is None:
            klass = type(obj)

        def newfunc(*args):
            return self.f(klass, *args)

        return newfunc

In [ ]:
from __future__ import annotations


from types import MethodType


def Attribute_Factory(cls):
    print(f"{cls=}, {dir(cls)=}")

    class Attribute:
        def __init__(self, func):
            self.func = func

        def __get__(self):
            return MethodType(self.f, cls)


#             @wraps(self.func)
#             def wrapper(*args, **kwargs):
#                 return self.func(klass, *args, **kwargs)

#         return wrapper

In [ ]:
# class demo:
#     __name__

#     @classmethod
#     def theclass(cls):
#         return cls

#     @Attribute_Factory(theclass)
#     def func(cls):
#         return cls.__name__

### LazyClassAttribute

In [ ]:
class A:
    @classmethod
    @property
    def myname(cls):
        return f"My name is {cls.__name__}!"

    @property
    def old_regular(self):
        return 42

In [ ]:
A.__dict__["myname"]

## Classmethod / Property Mixin

idea: use MethodType to do the dirty work!

In [ ]:
class Property:
    "Emulate PyProperty_Type() in Objects/descrobject.c"

    def __init__(self, fget=None, fset=None, fdel=None, doc=None):
        self.fget = fget
        self.fset = fset
        self.fdel = fdel
        if doc is None and fget is not None:
            doc = fget.__doc__
        self.__doc__ = doc

    def __get__(self, obj, objtype=None):
        if obj is None:
            return self
        if self.fget is None:
            raise AttributeError("unreadable attribute")
        return self.fget(obj)

    def __set__(self, obj, value):
        if self.fset is None:
            raise AttributeError("can't set attribute")
        self.fset(obj, value)

    def __delete__(self, obj):
        if self.fdel is None:
            raise AttributeError("can't delete attribute")
        self.fdel(obj)

    def getter(self, fget):
        return type(self)(fget, self.fset, self.fdel, self.__doc__)

    def setter(self, fset):
        return type(self)(self.fget, fset, self.fdel, self.__doc__)

    def deleter(self, fdel):
        return type(self)(self.fget, self.fset, fdel, self.__doc__)

In [ ]:
from types import MethodType


class ClassMethod:
    "Emulate PyClassMethod_Type() in Objects/funcobject.c"

    def __init__(self, f):
        self.f = f

    def __get__(self, obj, cls=None):
        if cls is None:
            cls = type(obj)
        if hasattr(type(self.f), "__get__"):
            return self.f.__get__(cls)
        return MethodType(self.f, cls)

In [ ]:
class Testit:
    @classmethod
    @property
    def clsprop(cls):
        """Real Class-Property"""
        return f"My name is {cls.__name__}"

    @ClassMethod
    @Property
    def myclsprop(cls):
        """My Class-Property"""
        return f"My name is {cls.__name__}"


print(Testit.clsprop, Testit.myclsprop)
print(Testit.__dict__)
help(Testit)